In [2]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import re

from utils.prepare_data import save_data, read_data

In [3]:
results = [Path(p) for p in snakemake.input]
result_dfs = []

for res in results:
    this_df = pd.read_table(res)
    target, training, _, rep_str = res.stem.split('_')
    replicate = int(rep_str.split('-')[-1])
    this_df = this_df.assign(replicate=replicate)
    result_dfs.append(this_df)

In [4]:
df = pd.concat(result_dfs)
df['locus'] = 'stc1'
df = df.dropna(axis='rows', how='any', subset=['locus'])
df = df.sort_values(by='locus').drop('uuid', axis='columns').dropna(axis='columns', how='all')
if 'predicted_log_selection_coefficient' in df.columns:
    df = (
        df
        .assign(predicted_selection_coefficient=10**df.predicted_log_selection_coefficient)
        .drop('predicted_log_selection_coefficient', axis='columns')
    )
df = df.pivot(index='replicate', columns='locus').reset_index()

In [10]:
save_data(df, snakemake.output["replicates_results"])
df.describe().drop('replicate', axis='columns', level=0).to_csv(snakemake.output["replicates_statistics"], sep='\t', index=True)